In [7]:
import numpy as np
import pandas as pd
import re # regular expression used for documents
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer # to convert words to numbers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None)

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

1# Preprocessing the data

In [10]:
from operator import index
news_dataset = pd.read_csv('train.csv')
news_dataset.shape


(20800, 5)

In [11]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [13]:
news_dataset = news_dataset.fillna('')
news_dataset.isna().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [14]:
# Merge Title and autore => content

news_dataset['content'] = news_dataset['author'] + news_dataset['title']

news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard PortnoyIranian woman jailed for fiction...


In [15]:
# Spliting the data
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

# Stemming > stemming is a process of a reducing a word to root word

In [16]:
port_stem = PorterStemmer()
def stemming (content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split() # converting to list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [17]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [18]:
news_dataset['content']

,content
0,darrel lucushous dem aid even see comey letter...
1,daniel j flynnflynn hillari clinton big woman ...
2,consortiumnew comwhi truth might get fire
3,jessica purkiss civilian kill singl us airstri...
4,howard portnoyiranian woman jail fiction unpub...
...,...
20795,jerom hudsonrapp trump poster child white supr...
20796,benjamin hoffmann f l playoff schedul matchup ...
20797,michael j de la merc rachel abramsmaci said re...
20798,alex ansarynato russia hold parallel exercis b...


In [19]:
X = news_dataset['content'].values #to returns values as a NumPy array
Y = news_dataset['label'].values
print(X,Y)

['darrel lucushous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exercis balkan'
 'david swansonwhat keep f aliv'] [1 0 1 ... 0 1 1]


In [20]:
Y.shape

(20800,)

In [21]:
# convert words to numbers
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)
print(X)

  (0, 26340)	0.28088379401596425
  (0, 22724)	0.2552336018069161
  (0, 15019)	0.43006226759639316
  (0, 14555)	0.29177259684200296
  (0, 12782)	0.24619727512767195
  (0, 8022)	0.23133661742488731
  (0, 6273)	0.2839932825877813
  (0, 5969)	0.35488202138141456
  (0, 5006)	0.2472595823572816
  (0, 4211)	0.3625320323150658
  (0, 578)	0.2694167078545385
  (1, 27923)	0.36911845953845024
  (1, 11313)	0.24166773097712638
  (1, 8772)	0.5258635625386451
  (1, 5916)	0.31810058109638056
  (1, 4767)	0.23338756776626793
  (1, 3859)	0.45980466668763476
  (1, 3281)	0.18652439327549428
  (1, 2622)	0.3562953366945267
  (2, 26235)	0.3665032495181434
  (2, 16361)	0.43295215406038445
  (2, 9454)	0.30743020569262086
  (2, 8567)	0.3411947414020896
  (2, 5240)	0.40440534260277944
  (2, 5121)	0.5511414848555652
  :	:
  (20797, 25776)	0.08220218573989037
  (20797, 25319)	0.3119640221826561
  (20797, 22086)	0.24902354987792552
  (20797, 20778)	0.2729578683228216
  (20797, 20493)	0.249994989010826
  (20797, 17505

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [23]:
model = LogisticRegression()

In [24]:
model.fit(X_train, Y_train)

LogisticRegression()

In [25]:
#testing prediction
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

testing_data_accuracy


0.9548076923076924

In [26]:
#training prediction
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

training_data_accuracy

0.9719951923076923

In [27]:
# predictive model
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [29]:
# Kaggle submission
submission_dataset = pd.read_csv('test.csv')
submission_dataset = submission_dataset.fillna('')

submission_dataset.head()


,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [30]:
submission_dataset['content'] = submission_dataset['author'] + submission_dataset['title']
submission_dataset['content'] = submission_dataset['content'].apply(stemming)
submission_dataset['content'].head()


,content
0,david streitfeldspect trump loosen tongu purs ...
1,russian warship readi strike terrorist near al...
2,common dream nodapl nativ american leader vow ...
3,daniel victortim tebow attempt anoth comeback ...
4,truth broadcast networkkeis report meme war e


In [31]:
X1= submission_dataset['content'].values
X1 = vectorizer.transform(X1)
Y_label = model.predict(X1)
print(Y_label)


[0 1 1 ... 0 1 0]


In [32]:
output = pd.DataFrame({'id': submission_dataset['id'], 'label': Y_label})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Try with other models

In [43]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsTransformer
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
!pip install torch # correct the package name
import torch # correct the module name

In [44]:
models= [DecisionTreeClassifier(), SVC(), RandomForestClassifier()] # Use classification models instead of regression models
for model in models:
  model.fit(X_train, Y_train)
  prediction = model.predict(X_test)
  print(model, accuracy_score(prediction, Y_test))

DecisionTreeClassifier() 0.9747596153846154
SVC() 0.9644230769230769
RandomForestClassifier() 0.9762019230769231


In [45]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, Y_train)
prediction = xgb.predict(X_test)
print(accuracy_score(prediction, Y_test))

0.9771634615384616


In [46]:
Y_label = xgb.predict(X1)
output = pd.DataFrame({'id': submission_dataset['id'], 'label': Y_label})
output.to_csv('submission1.csv', index=False)
print("Your submission was successfully saved!")